In [1]:
from symbotics.robots import Biped
import casadi as cs
import numpy as np 

biped_urdf = '../../robots_managerie/gr1/gr1.urdf'

biped_model = Biped(biped_urdf,
                    torso = {'torso': 'pelvis'},
                    foots = {'left_foot': 'footLeftY',
                            'right_foot': 'footRightY'},
                    arms = {'left_arm': 'wristRollLeft',
                           'right_arm': 'wristRollRight'})

In [4]:
# biped_model.joint_names
# biped_model.joint_map

In [2]:
jacobians = []
jacobians_dt = []
for body in {'right_foot', 'left_foot'}:
    contact_dim = getattr(biped_model, body).contact.dim
    jacobians.append(getattr(biped_model, body).contact.jacobian(biped_model._model._q))
    jacobians_dt.append(getattr(biped_model, body).jacobian_dt.world_aligned(biped_model._model._q, biped_model._model._v)[:contact_dim, :])

jacobian_matrix = cs.vertcat(*jacobians)
jacobian_dt = cs.vertcat(*jacobians_dt)

In [3]:
M = biped_model.inertia(biped_model._model._q) 
h = biped_model.bias_force(biped_model._model._q,biped_model._model._v)

In [4]:
nj = jacobian_matrix.rows()

In [5]:
# 
kkt_matrix = cs.blockcat(M, -jacobian_matrix.T, 
                         jacobian_matrix, np.zeros((nj, nj)))
kkt_rhs = cs.vertcat(biped_model._qfrc_u-h,-jacobian_dt @ biped_model._model._v)


In [6]:
kkt_solution = cs.solve(kkt_matrix, kkt_rhs)

In [7]:
ddq_minimal = kkt_solution[:biped_model.nv]
contact_forces = kkt_solution[biped_model.nv:]


In [8]:
fw_dyn = cs.Function("reduced_forward_dynamics",
            [biped_model._model._q,biped_model._model._v, biped_model._u],
            [ddq_minimal],
            ["q", "v", "u"],
            ["ddq_reduced"],
        )

In [9]:
con_force = cs.Function("contact_forces",
            [biped_model._model._q,biped_model._model._v, biped_model._u],
            [contact_forces],
            ["q", "v", "u"],
            ["constraint_forces"],
        )

In [10]:
con_force

Function(contact_forces:(q[30],v[29],u[23])->(constraint_forces[12]) SXFunction)

In [11]:
biped_model.state_space._state

SX([q_0, q_1, q_2, q_3, q_4, q_5, q_6, q_7, q_8, q_9, q_10, q_11, q_12, q_13, q_14, q_15, q_16, q_17, q_18, q_19, q_20, q_21, q_22, q_23, q_24, q_25, q_26, q_27, q_28, q_29, v_0, v_1, v_2, v_3, v_4, v_5, v_6, v_7, v_8, v_9, v_10, v_11, v_12, v_13, v_14, v_15, v_16, v_17, v_18, v_19, v_20, v_21, v_22, v_23, v_24, v_25, v_26, v_27, v_28])

In [12]:
fw_dyn(biped_model._model._q, biped_model._model._v, biped_model._u)*cs.SX.sym('dt', 1)

SX(@1=45.3937, @2=0.709978, @3=0.591505, @4=0.735108, @5=sin(q_19), @6=0.972965, @7=cos(q_20), @8=0.820041, @9=0.03965, @10=0.755431, @11=0.323747, @12=-0.03965, @13=cos(q_26), @14=0.717794, @15=0.032, @16=0.741361, @17=-0.032, @18=cos(q_28), @19=0.677185, @20=-0.000288809, @21=cos(q_29), @22=-0.149107, @23=sin(q_29), @24=((@20*@21)-(@22*@23)), @25=(-0.000188253+(@19*@24)), @26=0.000682642, @27=sin(q_28), @28=(@17+((@18*@25)+(@26*@27))), @29=(-0.00677959+(@16*@28)), @30=(@15+@29), @31=(0.0014789+(@14*@30)), @32=sin(q_26), @33=0.906308, @34=cos(q_27), @35=-0.422618, @36=sin(q_27), @37=((@33*@34)+(@35*@36)), @38=-0.13653, @39=-0.11, @40=(@39+((@20*@23)+(@22*@21))), @41=(-0.027911+(@19*@40)), @42=(@38+@41), @43=(-0.0245292+(@16*@42)), @44=((@35*@34)-(@33*@36)), @45=0.04, @46=(@45+((@26*@18)-(@27*@25))), @47=(0.00934636+(@16*@46)), @48=((@37*@43)+(@44*@47)), @49=(-2.46084e-06+(@14*@48)), @50=(@12+((@13*@31)+(@32*@49))), @51=(-0.0348031+(@11*@50)), @52=0.244569, @53=cos(q_22), @54=0.717794,

In [14]:
def func(q, v, u, dt):
    dv = fw_dyn(q, v, u)
    v_next = v + dv*dt
    # print(v_next)
    q_next = biped_model._model._kindyn.integrate()(q, v_next*dt)
    return cs.vertcat(q_next, v_next)

dt = cs.SX.sym('dt', 1)

discrete_state_space = cs.Function("discrete_state_space",
            [biped_model.state_space._state, biped_model._u, dt],
            [func(biped_model._model._q, biped_model._model._v, biped_model._u, dt)],
            ["state", "control", 'dt'],
            ["next_state"])

In [15]:
discrete_state_space

Function(discrete_state_space:(state[59],control[23],dt)->(next_state[59]) SXFunction)

In [16]:
%timeit discrete_state_space()

156 µs ± 3.41 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
%timeit fw_dyn()

142 µs ± 2.68 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
